In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Data Preprocess

In [9]:
# artist_names = ['Edgar Degas']
num_artists = 4

artist_df = pd.read_csv('/kaggle/input/best-artworks-of-all-time/artists.csv')
artist_df = artist_df.sort_values(by='paintings', ascending=False)
artist_df = artist_df.iloc[:num_artists]
# artist_df = artist_df[artist_df['name'].isin(artist_names)]
artist_list = artist_df['name'].str.replace(' ', '_').tolist()
artist_dict = {}
for i, n in enumerate(artist_list):
    artist_dict[n] = i
    
artist_dict

In [10]:
artist_df.head()

In [11]:
preprocessed_dir = '/kaggle/output/preprocessed'
image_dir = '/kaggle/input/best-artworks-of-all-time/images/images'

def preprocess(artist_name, image_size):
    """
    preprocess images into new image size and save them into corresponding dirs
    """
    artist_image_dir = os.path.join(image_dir, artist_name)
    artist_preprocessed_dir = os.path.join(preprocessed_dir, artist_name)

    !rm -rf {artist_preprocessed_dir}
    if not os.path.exists(artist_preprocessed_dir):
        os.makedirs(artist_preprocessed_dir)

    image_list = []
    for dirname, _, filenames in os.walk(artist_image_dir):
        for filename in filenames:
            image = Image.open(os.path.join(dirname, filename)).convert('RGB')
            image = image.resize(image_size)
            image_list.append(np.asarray(image))
            # image.save(os.path.join(artist_preprocessed_dir, filename))
    return image_list


image_size = (128, 128)

image_list = []
label_list = []
for artist_name in artist_list:
    il = preprocess(artist_name, image_size)
    image_list += il
    label_list += [artist_dict[artist_name] for i in range(len(il))]
    
image_list = np.array(image_list)
image_list = (image_list - 127.5) / 127.5
label_list = np.array(label_list)
assert len(image_list) == len(label_list)
print(len(image_list))

In [12]:
def display_artworks(image_list):
    """
    display artworks in 6*6 format
    """
    fig = plt.figure(figsize=(8,8))
    for i in range(1,37):
        fig.add_subplot(6,6,i)
        plt.imshow(image_list[i])
        plt.axis('off')

print(len(image_list))
display_artworks(image_list)

## Prepare Dataset

In [13]:
from tensorflow.keras import layers
import time
from IPython import display
import tensorflow as tf
tf.__version__

In [14]:
train_images = image_list
train_labels = label_list

BATCH_SIZE = 64
BUFFER_SIZE = 1000
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Define Models

In [15]:
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Conv2D, Conv2DTranspose, Reshape, Flatten
from keras.layers import Dropout, LeakyReLU, BatchNormalization
from keras.layers import Activation, ZeroPadding2D, UpSampling2D
from keras.layers import Input, Reshape

In [1]:
def make_generator_model():
    """
    definition of generator model
    """
    model = Sequential()

    model.add(Dense(128*128,activation="relu",input_dim=100)) #128x128 units
    model.add(Reshape((8,8,256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    model.add(UpSampling2D(size=(2,2)))
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D(size=(2,2)))
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(Conv2D(3,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))
    
    return model

In [18]:
def make_discriminator_model():
    """
    definition of discriminator model
    """
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[128, 128, 3]))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(0.2))
    
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    
    return model

In [19]:
generator = make_generator_model()

# random noise for generation
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0,:,:,0],interpolation='nearest')

In [20]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print(decision)

In [21]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    """
    discriminator loss function of cross entropy
    """
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    """
    generator loss function of cross entropy
    """
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4,0.5)

## Training

In [22]:
!rm -rf ./training_checkpoints
!mkdir ./training_checkpoints
!rm -rf ./image_at_epoch*


# checkpoints for model trianing, save models every certain epochs
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [23]:
EPOCHS = 500
noise_dim = 100
num_examples_to_generate = 4

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [24]:
@tf.function
def train_step(images):
    """
    train step for loss calculation and generation
    """
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    

In [25]:
def generate_and_save_images(model, epoch, test_input):
    """
    generate, display, and save images
    """
    predictions = model(test_input, training=False)
    predictions = 0.5 * predictions + 0.5

    fig = plt.figure(figsize=(32, 32))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [26]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      t = train_step(image_batch)

    display.clear_output(wait=True)
    
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [ ]:
train(train_dataset, 2)

## Generate Images

In [ ]:
seed = tf.random.normal([20, noise_dim])
predictions = generator(seed, training=False)
predictions = 0.5 * predictions + 0.5

fig = plt.figure(figsize=(10,10))
for i in range(1,21):
    fig.add_subplot(5,5,i)
    plt.imshow(predictions[i])
    plt.axis('off')

In [ ]:
!rm -rf ./generated_images
!mkdir ./generated_images

seed = tf.random.normal([200, noise_dim])
predictions = generator(seed, training=False)
predictions = 0.5 * predictions + 0.5

fig = plt.figure(figsize=(10,10))
for i in range(1,200):
    plt.axis('off')
    plt.imshow(generated_images2[i])
    plt.savefig('./images/%d.jpg' % i)